In [1]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

from matplotlib import pyplot as plt

from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

In [2]:
# ==========
# helping methods
# ==========

# generate a list from lower and upper bound
def gen_list(p0, pn, delta, dig=5):
    ret = []
    i = p0
    while i < pn:
        ret.append(float(i))
        i += delta
        i = round(i, dig)
    return ret

# padding and zero padding
def padding(origin, a_list, b_list):
    return np.hstack((a_list, origin, b_list))

def zero_padding(origin, num):
    zero_list = [0 for i in range(num)]
    return padding(origin, zero_list, zero_list)

# trainning pairs
def gen_pair(u, x, t, length=3, num=1000):
    pairs = []
    for i in range(num):
        r = random.randint(0, t-2)
        current_t = u[r]
        next_t = u[r+1]
        p = random.randint(length, x-1-length)
        train = current_t[p-length:p+length+1]
        solu = next_t[p]
        pair = {'input': train, 'solu': solu}
        pairs.append(pair)
    return pairs

# trainning pairs (in average)
def gen_pair_ave(u, x, t, length=3, num=1000):
    pairs = []
    for i in range(num):
        r = random.randint(0, t-2)
        current_t = u[r]
        next_t = u[r+1]
        p = random.randint(length, x-2-length)
        train0 = current_t[p-length-1:p+length+2]
        solu1 = next_t[p]
        solu2 = next_t[p+1]
        train = []
        for j in range(len(train0)-1):
                train.append(0.5 * (train0[j] + train0[j+1]))
        solu = 0.5 * (solu1 + solu2)
        pair = {'input': train, 'solu': solu}
        pairs.append(pair)
    return pairs

In [3]:
# the analytical representation of exact solution
def heat_equ_analytical_solu(x, t):
    return np.sin(np.pi * x) * np.exp(-np.power(np.pi, 2) * t)

In [4]:
# restnet
class ResNet(nn.Module):
    def __init__(self, i, h, o):
        super(ResNet, self).__init__()
        self.linear1 = nn.Linear(i, h)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(h, o)
        self.relu2 = nn.ReLU()
        
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu1(out)
        out = self.linear2(out)
        out = self.relu2(out)
        return out + torch.mean(x)

    def load_model(self, save_path):
        self.load_state_dict(torch.load(save_path))

    def save_model(self, save_path):
        torch.save(self.state_dict(), save_path)

In [5]:
def calc_next_time(current, model, length=3):
    ret = []
    p = np.hstack(([0 for i in range(length)], current, [0 for i in range(length)])).tolist()
    for index in range(len(current)):
        i = index + 3
        seg = p[i-length:i+length+1]
        tensor_seg = torch.FloatTensor(seg)
        out = model(tensor_seg)
        ret.append(out.item())
    return ret

## $\Delta x=\frac{1}{20}$ $\Delta t=\frac{1}{20}$

In [6]:
x_1 = arange(0, 2 * np.pi, 1/20)
t_1 = arange(0, 2 * np.pi, 1/20)
X_1,T_1 = meshgrid(x_1, t_1) # grid of point
Z_1 = heat_equ_analytical_solu(X_1, T_1) # evaluation of the function on the grid

In [7]:
padding_1 = []
for z in Z_1:
    p = np.hstack(([0,0,0], z, [0,0,0]))
    padding_1.append(p.tolist())

In [8]:
model_1 = ResNet(7, 6, 1)
optimizer_1 = optim.Adam(model_1.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
criterion_1 = nn.MSELoss()
model_1.train()

ResNet(
  (linear1): Linear(in_features=7, out_features=6, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=6, out_features=1, bias=True)
  (relu2): ReLU()
)

In [9]:
model_1.load_model("model_xx delta x=1 20 delta t=1 20")

In [10]:
prediction_1 = []
prediction_1.append(torch.FloatTensor(Z_1[0]).tolist())
for i in range(len(Z_1)-1):
    prediction_1.append(calc_next_time(Z_1[i], model_1))
    print("time", i, "is done", '", an example is:"', prediction_1[i+1][0])

time 0 is done ", an example is:" 0.1313488483428955
time 1 is done ", an example is:" 0.08018821477890015
time 2 is done ", an example is:" 0.04895474761724472
time 3 is done ", an example is:" 0.029886778444051743
time 4 is done ", an example is:" 0.018245818093419075
time 5 is done ", an example is:" 0.01113903522491455
time 6 is done ", an example is:" 0.006800359580665827
time 7 is done ", an example is:" 0.004151606000959873
time 8 is done ", an example is:" 0.0025345473550260067
time 9 is done ", an example is:" 0.0015473359962925315
time 10 is done ", an example is:" 0.0009446456097066402
time 11 is done ", an example is:" 0.0005767042748630047
time 12 is done ", an example is:" 0.00035207680775783956
time 13 is done ", an example is:" 0.00021494222164619714
time 14 is done ", an example is:" 0.0001312217937083915
time 15 is done ", an example is:" 8.011063619051129e-05
time 16 is done ", an example is:" 4.8907390009844676e-05
time 17 is done ", an example is:" 2.98578652291325

In [11]:
diff_1 = (prediction_1-Z_1).tolist()

In [12]:
import csv

with open("prediction_11 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(prediction_1)

In [13]:
import csv

with open("diff_11 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(diff_1)

## $\Delta x=\frac{1}{40}$ $\Delta t=\frac{1}{20}$

In [14]:
x_2 = arange(0, 2 * np.pi, 1/40)
t_2 = arange(0, 2 * np.pi, 1/20)
X_2,T_2 = meshgrid(x_2, t_2) # grid of point
Z_2 = heat_equ_analytical_solu(X_2, T_2) # evaluation of the function on the grid

In [15]:
padding_2 = []
for z in Z_2:
    p = np.hstack(([0,0,0], z, [0,0,0]))
    padding_2.append(p.tolist())

In [16]:
model_2 = ResNet(7, 6, 1)
optimizer_2 = optim.Adam(model_2.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
criterion_2 = nn.MSELoss()
model_2.train()

ResNet(
  (linear1): Linear(in_features=7, out_features=6, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=6, out_features=1, bias=True)
  (relu2): ReLU()
)

In [17]:
model_2.load_model("model_xx delta x=1 40 delta t=1 20")

In [18]:
prediction_2 = []
prediction_2.append(torch.FloatTensor(Z_2[0]).tolist())
for i in range(len(Z_2)-1):
    prediction_2.append(calc_next_time(Z_2[i], model_2))
    print("time", i, "is done", '", an example is:"', prediction_2[i+1][0])

time 0 is done ", an example is:" 0.06690555810928345
time 1 is done ", an example is:" 0.04084571450948715
time 2 is done ", an example is:" 0.02493622712790966
time 3 is done ", an example is:" 0.015223517082631588
time 4 is done ", an example is:" 0.009293926879763603
time 5 is done ", an example is:" 0.005673923995345831
time 6 is done ", an example is:" 0.0034639197401702404
time 7 is done ", an example is:" 0.0021147162187844515
time 8 is done ", an example is:" 0.0012910299701616168
time 9 is done ", an example is:" 0.0007881712517701089
time 10 is done ", an example is:" 0.00048117697588168085
time 11 is done ", an example is:" 0.0002982875448651612
time 12 is done ", an example is:" 0.00022809502843301743
time 13 is done ", an example is:" 0.00018524326151236892
time 14 is done ", an example is:" 0.00015907903434708714
time 15 is done ", an example is:" 0.00014310267579276115
time 16 is done ", an example is:" 0.00013334785762708634
time 17 is done ", an example is:" 0.0001273

In [19]:
diff_2 = (prediction_2-Z_2).tolist()

In [20]:
import csv

with open("prediction_21 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(prediction_2)

In [21]:
import csv

with open("diff_21 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(diff_2)

## $\Delta x=\frac{1}{80}$ $\Delta t=\frac{1}{20}$

In [22]:
x_3 = arange(0, 2 * np.pi, 1/80)
t_3 = arange(0, 2 * np.pi, 1/20)
X_3,T_3 = meshgrid(x_3, t_3) # grid of point
Z_3 = heat_equ_analytical_solu(X_3, T_3) # evaluation of the function on the grid

In [23]:
padding_3 = []
for z in Z_3:
    p = np.hstack(([0,0,0], z, [0,0,0]))
    padding_3.append(p.tolist())

In [24]:
model_3 = ResNet(7, 6, 1)
optimizer_3 = optim.Adam(model_3.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
criterion_3 = nn.MSELoss()
model_3.train()

ResNet(
  (linear1): Linear(in_features=7, out_features=6, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=6, out_features=1, bias=True)
  (relu2): ReLU()
)

In [25]:
model_3.load_model("model_xx delta x=1 80 delta t=1 20")

In [26]:
prediction_3 = []
prediction_3.append(torch.FloatTensor(Z_3[0]).tolist())
for i in range(len(Z_3)-1):
    prediction_3.append(calc_next_time(Z_3[i], model_3))
    print("time", i, "is done", '", an example is:"', prediction_3[i+1][0])

time 0 is done ", an example is:" 0.03360804542899132
time 1 is done ", an example is:" 0.02051764354109764
time 2 is done ", an example is:" 0.012525982223451138
time 3 is done ", an example is:" 0.00764708686619997
time 4 is done ", an example is:" 0.004668531473726034
time 5 is done ", an example is:" 0.002850129036232829
time 6 is done ", an example is:" 0.00173999834805727
time 7 is done ", an example is:" 0.0010622654808685184
time 8 is done ", an example is:" 0.0006485110498033464
time 9 is done ", an example is:" 0.00039591468521393836
time 10 is done ", an example is:" 0.00024170511460397393
time 11 is done ", an example is:" 0.00014756050950381905
time 12 is done ", an example is:" 9.008539927890524e-05
time 13 is done ", an example is:" 5.4996955441311e-05
time 14 is done ", an example is:" 3.3575535780983046e-05
time 15 is done ", an example is:" 2.049779868684709e-05
time 16 is done ", an example is:" 1.2513864021457266e-05
time 17 is done ", an example is:" 7.639690011274

In [27]:
diff_3 = (prediction_3-Z_3).tolist()

In [28]:
import csv

with open("prediction_31 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(prediction_3)

In [29]:
import csv

with open("diff_31 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(diff_3)

## $\Delta x=\frac{1}{160}$ $\Delta t=\frac{1}{20}$

In [30]:
x_4 = arange(0, 2 * np.pi, 1/160)
t_4 = arange(0, 2 * np.pi, 1/20)
X_4,T_4 = meshgrid(x_4, t_4) # grid of point
Z_4 = heat_equ_analytical_solu(X_4, T_4) # evaluation of the function on the grid

In [31]:
padding_4 = []
for z in Z_4:
    p = np.hstack(([0,0,0], z, [0,0,0]))
    padding_4.append(p.tolist())

In [32]:
model_4 = ResNet(7, 6, 1)
optimizer_4 = optim.Adam(model_4.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
criterion_4 = nn.MSELoss()
model_4.train()

ResNet(
  (linear1): Linear(in_features=7, out_features=6, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=6, out_features=1, bias=True)
  (relu2): ReLU()
)

In [33]:
model_4.load_model("model_xx delta x=1 160 delta t=1 20")

In [34]:
prediction_4 = []
prediction_4.append(torch.FloatTensor(Z_4[0]).tolist())
for i in range(len(Z_4)-1):
    prediction_4.append(calc_next_time(Z_4[i], model_4))
    print("time", i, "is done", '", an example is:"', prediction_4[i+1][0])

time 0 is done ", an example is:" 0.01682347245514393
time 1 is done ", an example is:" 0.010270697996020317
time 2 is done ", an example is:" 0.006270240526646376
time 3 is done ", an example is:" 0.0038279693108052015
time 4 is done ", an example is:" 0.0023369677364826202
time 5 is done ", an example is:" 0.0014267141232267022
time 6 is done ", an example is:" 0.0008710061083547771
time 7 is done ", an example is:" 0.0005317475297488272
time 8 is done ", an example is:" 0.00032463084789924324
time 9 is done ", an example is:" 0.00019818647706415504
time 10 is done ", an example is:" 0.00012099245941499248
time 11 is done ", an example is:" 7.38656526664272e-05
time 12 is done ", an example is:" 4.509483915171586e-05
time 13 is done ", an example is:" 2.753030821622815e-05
time 14 is done ", an example is:" 1.68071983353002e-05
time 15 is done ", an example is:" 1.0260760973324068e-05
time 16 is done ", an example is:" 6.264174771786202e-06
time 17 is done ", an example is:" 3.824266

In [35]:
diff_4 = (prediction_4-Z_4).tolist()

In [36]:
import csv

with open("prediction_41 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(prediction_4)

In [37]:
import csv

with open("diff_41 xx.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(diff_4)